# Computer Vision, Fall 2022

## Exercise 5

## Ex 5.1. Camera matrix (4 p)

Using the focal length, principal point and skew coefficient values given in the template, and equations given in Lecture 6, slide 49, form the camera matrix P and compute image coordinates (x,y) for an object point X (0, 0, 1, 1) when the origin of the world coordinate frame is

- exactly 3 meters away from the camera center, i.e. t = (0, 0, 3), and the camera is completely aligned with the world coordinate axis
- exactly 5 meters away from the camera center, i.e. t = (0, 0, 5), and the camera is completely aligned with the world coordinate axis
- t = (0.5, 1, 3) and the camera otherwise aligned with the world coordinate axis but it has only turned 20 degrees to the left
- t = (15, 1, 3) and the camera otherwise aligned with the world coordinate axis but it has only turned 20 degrees to the left

Discuss the phenomena behind the results.

In [1]:
import numpy as np
import cv2 as cv

# Focal length
fc = [719.302047058490760, 718.392548175289110]

# Principal point
cc = [334.631234942930060, 256.166677783686790]

# Skew coefficient
alpha_c = 0.000000000000000


# Your solution here
# Object point
X = np.array([0, 0, 1, 1])

# Scenarios
scenarios = [
    {'t': np.array([0, 0, 3]), 'rotation_matrix': np.eye(3)},
    {'t': np.array([0, 0, 5]), 'rotation_matrix': np.eye(3)},
    {'t': np.array([0.5, 1, 3]), 'rotation_matrix': np.array([[1, 0, 0], [0, np.cos(np.radians(20)), -np.sin(np.radians(20))], [0, np.sin(np.radians(20)), np.cos(np.radians(20))]])},
    {'t': np.array([15, 1, 3]), 'rotation_matrix': np.array([[1, 0, 0], [0, np.cos(np.radians(20)), -np.sin(np.radians(20))], [0, np.sin(np.radians(20)), np.cos(np.radians(20))]])}
]

for i, scenario in enumerate(scenarios):
    t = scenario['t']
    R = scenario['rotation_matrix']

    # Form the camera matrix P
    K = np.array([[fc[0], alpha_c, cc[0]],
                  [0, fc[1], cc[1]],
                  [0, 0, 1]])
    T = np.hstack((R, t.reshape(-1, 1)))
    P = np.dot(K, T)

    # Compute image coordinates (x, y)
    Xc = np.dot(R, X[:3]) + t
    x = (Xc[0] / Xc[2]) * fc[0] + cc[0]
    y = (Xc[1] / Xc[2]) * fc[1] + cc[1]

    print(f"Scenario {i+1}:")
    print("Camera Matrix P:")
    print(P)
    print(f"Image Coordinates (x, y): ({x:.2f}, {y:.2f})\n")
    
#In Scenario 1 and Scenario 2, Camera Aligned with the World Coordinate Axis and at Different Distances.
#The camera is aligned with the world coordinate axis and is positioned at
#different distances from the object point (3 meters in Scenario 1 and 5 meters in Scenario 2). 
#Since the camera is aligned with the world coordinates and no rotation is applied, the image 
#coordinates remain constant. This demonstrates that the camera's position relative to the object
#has a direct impact on the scaling of the image coordinates.

#In Scenario 3 and Scenario 4,Camera Turned to the Left and at Different Distances .The camera is not 
#aligned with the world coordinate axis. It has been turned 20 degrees to the left. The effect
#of the camera rotation is evident in these results. The image coordinates(x, y) are significantly
#different from the previous scenarios. The impact of the rotation is more pronounced in Scenario 4, 
#where the camera is 15 meters away from the object.The camera rotation causes a change in
#the perspective, leading to different image coordinates.

Scenario 1:
Camera Matrix P:
[[7.19302047e+02 0.00000000e+00 3.34631235e+02 1.00389370e+03]
 [0.00000000e+00 7.18392548e+02 2.56166678e+02 7.68500033e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00 3.00000000e+00]]
Image Coordinates (x, y): (334.63, 256.17)

Scenario 2:
Camera Matrix P:
[[7.19302047e+02 0.00000000e+00 3.34631235e+02 1.67315617e+03]
 [0.00000000e+00 7.18392548e+02 2.56166678e+02 1.28083339e+03]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00 5.00000000e+00]]
Image Coordinates (x, y): (334.63, 256.17)

Scenario 3:
Camera Matrix P:
[[ 7.19302047e+02  1.14450623e+02  3.14450502e+02  1.36354473e+03]
 [ 0.00000000e+00  7.62682340e+02 -4.98678549e+00  1.48689258e+03]
 [ 0.00000000e+00  3.42020143e-01  9.39692621e-01  3.00000000e+00]]
Image Coordinates (x, y): (425.92, 376.15)

Scenario 4:
Camera Matrix P:
[[ 7.19302047e+02  1.14450623e+02  3.14450502e+02  1.17934244e+04]
 [ 0.00000000e+00  7.62682340e+02 -4.98678549e+00  1.48689258e+03]
 [ 0.00000000e+00  3.42020143e-01  9

## Ex 5.2 Fundamental matrix (4 p)

Compute the fundamental matrix F for the images image1.jpg and image2.jpg using the normalized 8-point algorithm. You may use ready made functions for detecting and matching the features, but develop the algorithm for deriving matrix F by yourself. Where are the epipoles for both images? Report both F and the two epipoles.

In [2]:
import numpy as np
import cv2

def F_8point(x1, x2):

# Your solution here
    # Normalize the points
    x1_normalized = x1 / np.linalg.norm(x1, axis=1)[:, np.newaxis]
    x2_normalized = x2 / np.linalg.norm(x2, axis=1)[:, np.newaxis]

    # Construct the A matrix
    A = np.column_stack((x1_normalized[:, 0] * x2_normalized[:, 0],
                         x1_normalized[:, 0] * x2_normalized[:, 1],
                         x1_normalized[:, 0],
                         x1_normalized[:, 1] * x2_normalized[:, 0],
                         x1_normalized[:, 1] * x2_normalized[:, 1],
                         x1_normalized[:, 1],
                         x2_normalized[:, 0],
                         x2_normalized[:, 1],
                         np.ones(len(x1_normalized))))

    # Solve for the fundamental matrix using SVD
    _, _, V = np.linalg.svd(A)
    F = V[-1].reshape(3, 3)

    # Enforce the rank-2 constraint on F
    U, S, Vt = np.linalg.svd(F)
    S[2] = 0
    F = np.dot(U, np.dot(np.diag(S), Vt))

    return F

# Load images
image1 = cv2.imread('image 1.jpg')
image2 = cv2.imread('image 2.jpg')


# Example using SIFT (replace with your preferred method):
sift = cv2.SIFT_create()

# Detect key points and compute descriptors for both images
kp1, des1 = sift.detectAndCompute(image1, None)
kp2, des2 = sift.detectAndCompute(image2, None)

# Perform feature matching
bf = cv2.BFMatcher()
matches = bf.knnMatch(des1, des2, k=2)

# Apply ratio test to obtain good matches
good_matches = []
for m, n in matches:
    if m.distance < 0.75 * n.distance:
        good_matches.append(m)

# Get corresponding feature points
x1 = np.array([kp1[match.queryIdx].pt + (1,) for match in good_matches])
x2 = np.array([kp2[match.trainIdx].pt + (1,) for match in good_matches])

# Compute the fundamental matrix F
F = F_8point(x1, x2)

# Compute the epipoles using SVD
U, S, Vt = np.linalg.svd(F)
epipole1 = Vt[-1]  
epipole2 = U[:, -1] 

print("Fundamental Matrix F:")
print(F)
print("Epipole in Image 1:")
print(epipole1)
print("Epipole in Image 2:")
print(epipole2)

Fundamental Matrix F:
[[-0.27143075 -0.4200483   0.445115  ]
 [-0.09650671 -0.16247703  0.16056809]
 [ 0.28130399  0.44379275 -0.46279422]]
Epipole in Image 1:
[-0.80295709 -0.10320342 -0.58703405]
Epipole in Image 2:
[0.56138292 0.44843658 0.69552416]
